ДЗ02 Бондаренко Алексей 
Поиск часто встречающихся похожих слов


In [ ]:
Подготовка текста

In [3]:
import re
from collections import Counter

with open ("qualia.txt", "r") as file:
    text = file.read()
text = text.lower()
text = re.sub('[^а-яё]+', ' ', text)

Разбиение текста на слова, подсчет слов

In [41]:
import numpy as np
import pandas as pd
words = text.split(' ')

total_words_count = len(words)
words = pd.Series(words)
unique_words_count = words.value_counts()
print(unique_words_count[:10])
print(total_words_count)

и        1679
не       1518
я        1357
в        1048
что       959
юкари     675
на        589
с         561
как       550
но        505
dtype: int64
45023


Первый десяток уникальных слов с количеством. 
Всего уникальных слов 9000, это очень много. Таблица сравнения 9000x9000 по моим подсчетам будет считаться час. Выведу количество редких слов


In [74]:
rare_count = lambda arr, count:  sum([1 if val==count else 0 for idx, val in unique_words_count.iteritems()])
print(rare_count(unique_words_count, 1))
print(rare_count(unique_words_count, 2))
print(rare_count(unique_words_count, 3))
print(rare_count(unique_words_count, 4))


5781
1255
587
294


сократим массив удалив все слова встречающиеся менее трех раз

In [75]:
drop_index_list = []
for idx, val in unique_words_count.iteritems():
    if val<3:
        drop_index_list.append(idx)
unique_words_count = unique_words_count.drop(labels=drop_index_list)
unique_words_list = list(unique_words_count.index)
len(unique_words_count)

2010

Создаю матрийу сравнения 2000х2000 и заполняю процентом похожести двух слов.
для проверки похожести использую difflib. У метода ratio() следующий алгоритм:
2.0*M / T где T количество элементов в обоих словах а M количество совпадений

In [31]:
%%time
import difflib
df = pd.DataFrame(data = np.zeros(shape=(len(unique_words_list),len(unique_words_list))),columns = unique_words_list, index = unique_words_list)
counter = 1
for i in unique_words_list:
    for j in unique_words_list:
        counter+=1
        if counter%500000 == 0:
            print(counter)
        df.loc[i,j] = difflib.SequenceMatcher(a=i, b=j).ratio()

500000
1000000
1500000
2000000
2500000
3000000
3500000
4000000
                       и        не         я         в       что     юкари  \
и               1.000000  0.000000  0.000000  0.000000  0.000000  0.333333   
не              0.000000  1.000000  0.000000  0.000000  0.000000  0.000000   
я               0.000000  0.000000  1.000000  0.000000  0.000000  0.000000   
в               0.000000  0.000000  0.000000  1.000000  0.000000  0.000000   
что             0.000000  0.000000  0.000000  0.000000  1.000000  0.000000   
юкари           0.333333  0.000000  0.000000  0.000000  0.000000  1.000000   
на              0.000000  0.500000  0.000000  0.000000  0.000000  0.285714   
с               0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
как             0.000000  0.000000  0.000000  0.000000  0.000000  0.500000   
но              0.000000  0.500000  0.000000  0.000000  0.400000  0.000000   
а               0.000000  0.000000  0.000000  0.000000  0.000000  0.333333   
т

9 минут это долговато поэтому сохраняю в файл результат для дальнейшей работы

In [34]:
df.to_csv('comparison_matrix.csv')

я выбрал коэфициент похожести в 0.75 так ниже не удовлетворял меня (и ~= ли 0.66) и пробегаясь по матрие сравнения - удаляю из результата похожие элементы

In [76]:
result = unique_words_count.copy()
comparison_matrix = pd.read_csv('comparison_matrix.csv', header=0, index_col=0)
comparison_matrix
for i in unique_words_list:
    for j in unique_words_list:
        if i in result and j in result and comparison_matrix[i][j] >= 0.75 and comparison_matrix[i][j] < 1:
            result.loc[i] = result[i] + result[j]
            del result[j]
result = result.sort_values()[::-1]
print(result)

не               2095
и                1679
что              1462
я                1357
в                1048
на               1028
как               696
юкари             675
так               619
это               589
с                 561
но                535
её                440
а                 406
было              396
же                351
по                325
меня              322
от                293
ты                281
всё               265
себя              246
только            245
когда             239
из                234
ни                229
у                 225
поэтому           219
друг              217
да                210
                 ... 
справлюсь           3
философский         3
прекрати            3
уехала              3
померещилось        3
описать             3
внутри              3
угадала             3
стыдно              3
интерпретация       3
золотые             3
вырезать            3
главным             3
одиночестве         3
мягкая    

Для интереса попробую удалить служебные части речи, при помощи библиотеки pymorphy2

In [ ]:
!pip install pymorphy2

In [80]:
import pymorphy2
additional = result.copy()
pm2 = pymorphy2.MorphAnalyzer()
drop_words=[]
for word in additional.index:
    if pm2.parse(word)[0].tag.POS in ['INTJ', 'PRCL', 'CONJ', 'PREP']:
        drop_words.append(word)
additional = additional.drop(drop_words)
additional = pd.DataFrame(additional, columns=['Count'])
additional['%'] = additional['Count'] / total_words_count
additional

,Count,%
я,1357,0.030140
юкари,675,0.014992
её,440,0.009773
было,396,0.008796
меня,322,0.007152
ты,281,0.006241
всё,265,0.005886
себя,246,0.005464
поэтому,219,0.004864
друг,217,0.004820


Если прочесть 30 первых слов то получается практически краткий пересказ книги))
